In [39]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim

In [40]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Hyper-parameters 
input_size = 30
hidden_size = 180
num_classes = 2
num_epochs = 20
learning_rate = 0.001
momentum = 0.9

In [41]:
class NeuralNet(nn.Module):
    
    '''Fully connected neural network with one hidden layer
    '''
    
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [42]:
# dataset
data = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.25, random_state=2020)
X_train, X_test = torch.tensor(X_train, dtype=torch.float), torch.tensor(X_test, dtype=torch.float)
y_train, y_test = torch.tensor(y_train, dtype=torch.long), torch.tensor(y_test, dtype=torch.long)

In [48]:
model = NeuralNet(input_size, hidden_size, num_classes)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum) 
for epoch in range(num_epochs):
    X_train.to(device)
    y_train.to(device)
    output = model(X_train)
    loss = criterion(output, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    print('Epoch: [{0}]/[{1}] loss: {2:.2f}'.format(epoch+1, num_epochs, loss))

Epoch: [1]/[20] loss: 9.97
Epoch: [2]/[20] loss: 595.24
Epoch: [3]/[20] loss: 282.53
Epoch: [4]/[20] loss: 0.98
Epoch: [5]/[20] loss: 0.49
Epoch: [6]/[20] loss: 0.39
Epoch: [7]/[20] loss: 0.43
Epoch: [8]/[20] loss: 0.38
Epoch: [9]/[20] loss: 0.45
Epoch: [10]/[20] loss: 0.30
Epoch: [11]/[20] loss: 0.42
Epoch: [12]/[20] loss: 0.29
Epoch: [13]/[20] loss: 0.36
Epoch: [14]/[20] loss: 0.29
Epoch: [15]/[20] loss: 0.34
Epoch: [16]/[20] loss: 0.27
Epoch: [17]/[20] loss: 0.32
Epoch: [18]/[20] loss: 0.24
Epoch: [19]/[20] loss: 0.29
Epoch: [20]/[20] loss: 0.23


In [49]:
with torch.no_grad():
    correct = 0
    total = 0
    X_test.to(device)
    y_test.to(device)
    outputs = model(X_test)
    _, predicted = torch.max(outputs.data, 1)
    total += y_test.size(0)
    correct += (predicted == y_test).sum().item()

    print('Accuracy of the network on the Breast Cancer dataset: {0} %'.format(100 * correct / total))

Accuracy of the network on the Breast Cancer dataset: 90.20979020979021 %


In [50]:
torch.save(model.state_dict(), 'dnn_model.pth')